########################################################################################################################################################################

In [1]:
import numpy as np
import pandas as pd
import datetime, re, gc, time, pickle, holidays
from countryinfo import CountryInfo
import os
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandasql as psql

# pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 500)

## Data Path
fold_pth = "~/ds7900_spring23_team2/"
output_pth = fold_pth + "output_data/"

email_hist_pth = os.path.expanduser(fold_pth + "email_hist/email_hist.csv")
stay_hist_pth = os.path.expanduser(fold_pth + "stay_hist/stay_hist.csv")
members_pth = os.path.expanduser(fold_pth + "members/members.csv")

members_pth_process = os.path.expanduser(output_pth + "members.csv")

stay_hist_pth_process = os.path.expanduser(output_pth + "stay_hist.csv")
stay_hist_pth_process2 = os.path.expanduser(output_pth + "stay_hist2.csv")

email_hist_pth_process = os.path.expanduser(output_pth + "email_hist.csv")
email_hist_pth_process2 = os.path.expanduser(output_pth + "email_hist_proc.csv")
email_hist_pth_process3 = os.path.expanduser(output_pth + "email_hist_proc_partial_stay.csv")
email_hist_pth_process4 = os.path.expanduser(output_pth + "email_hist_proc_full_stay.csv")

############################################################################################################################################################################################################################
############################################################################################################################################################################################################################

# Functions 

# Calculate holiday-related columns based on MBR_SUBREGION
def days_since_last_holiday(send_date, subregion, holiday_lookup_dict):
    holidays = holiday_lookup_dict.get(subregion, [])
    previous_holidays = [h for h in holidays if h < send_date]
    if previous_holidays:
        return (send_date - max(previous_holidays)).days
    else:
        return None

def days_until_next_holiday(send_date, subregion, holiday_lookup_dict):
    holidays = holiday_lookup_dict.get(subregion, [])
    next_holidays = [h for h in holidays if h > send_date]
    if next_holidays:
        return (min(next_holidays) - send_date).days
    else:
        return None
    
def translate_element(element, translation_dict):
    return translation_dict.get(element, element)

def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'
    
def get_nearest_holiday(send_date, subregion, holiday_lookup_dict):
    import datetime
    
    if type(send_date) != type(list()):
        holiday_list = [[(holiday_tup[0] - send_date.date()).days, holiday_tup[0], holiday_tup[1]] for holiday_tup in holiday_lookup_dict[subregion].items()]
        holiday_diff_list = [abs(h[0]) for h in holiday_list]
        ret_val = holiday_list[holiday_diff_list.index(np.min(holiday_diff_list))]
    
    if type(send_date) == type(list()):
        temp_dict = holiday_lookup_dict[subregion].items()
        ret_val = []
        for s in send_date:
            holiday_list = [[(holiday_tup[0] - s.date()).days, holiday_tup[0], holiday_tup[1]] for holiday_tup in holiday_lookup_dict[subregion].items()]
            holiday_diff_list = [abs(h[0]) for h in holiday_list]
            ret_val = ret_val + holiday_list[holiday_diff_list.index(np.min(holiday_diff_list))]
        
    return ret_val

def get_country_info(country_name, prefix = ""):
    from countryinfo import CountryInfo
    country_dict = {"MAINLAND CHINA": "CHINA", "MACEDONIA THE FORMER YUGOSLAV REPUBLIC OF": "GREECE", "LAO PEOPLES DEMOCRATIC REPUBLIC": "LAOS", "RUSSIAN FEDERATION": "RUSSIA", "SAUDI ARABIA": "SAUDI ARABIA", "SERBIA": "SERBIA", "SEYCHELLES": "SEYCHELLES", "SINGAPORE": "SINGAPORE", "SLOVAKIA": "SLOVAKIA", "SLOVENIA": "SLOVENIA", "SOUTH AFRICA": "SOUTH AFRICA", "SPAIN": "SPAIN", "SRI LANKA": "SRI LANKA", "SWEDEN": "SWEDEN", "SWITZERLAND": "SWITZERLAND", "TAIWAN": "TAIWAN", "TANZANIA UNITED REPUBLIC OF": "TANZANIA", "THAILAND": "THAILAND", "TRINIDAD AND TOBAGO": "TRINIDAD AND TOBAGO", "TURKEY": "TURKEY", "UKRAINE": "UKRAINE", "UNITED ARAB EMIRATES": "UNITED ARAB EMIRATES", "UNITED KINGDOM": "UNITED KINGDOM", "UNITED STATES": "UNITED STATES", "URUGUAY": "URUGUAY", "VANUATU": "VANUATU", "VENEZUELA BOLIVARIAN REPUBLIC OF": "VENEZUELA", "VIET NAM": "VIETNAM", "ZAMBIA": "ZAMBIA", "ZIMBABWE": "ZIMBABWE", "ALGERIA": "ALGERIA", "ANDORRA": "SPAIN", "ARGENTINA": "ARGENTINA", "ARMENIA": "ARMENIA", "ARUBA": "ARUBA", "AUSTRALIA": "AUSTRALIA", "AUSTRIA": "AUSTRIA", "AZERBAIJAN": "AZERBAIJAN", "BAHAMAS": "CUBA", "BAHRAIN": "BAHRAIN", "BANGLADESH": "BANGLADESH", "BELARUS": "BELARUS", "BELGIUM": "BELGIUM", "BRAZIL": "BRAZIL", "BULGARIA": "BULGARIA", "CANADA": "CANADA", "CAYMAN ISLANDS": "CAYMAN ISLANDS", "CHILE": "CHILE", "COLOMBIA": "COLOMBIA", "COSTA RICA": "COSTA RICA", "CROATIA": "CROATIA", "CYPRUS": "CYPRUS", "CZECH REPUBLIC": "CZECH REPUBLIC", "DENMARK": "DENMARK", "DOMINICAN REPUBLIC": "DOMINICAN REPUBLIC", "ECUADOR": "ECUADOR", "EGYPT": "EGYPT", "EL SALVADOR": "EL SALVADOR", "FIJI": "FIJI", "FINLAND": "FINLAND", "FRANCE": "FRANCE", "FRENCH POLYNESIA": "FRENCH POLYNESIA", "GEORGIA": "GEORGIA", "GERMANY": "GERMANY", "GHANA": "GHANA", "GIBRALTAR": "GIBRALTAR", "GREECE": "GREECE", "GUAM": "GUAM", "GUATEMALA": "GUATEMALA", "HONDURAS": "HONDURAS", "HONG KONG": "HONG KONG", "HUNGARY": "HUNGARY", "ICELAND": "ICELAND", "INDIA": "INDIA", "INDONESIA": "INDONESIA", "IRELAND": "IRELAND", "ISRAEL": "ISRAEL", "ITALY": "ITALY", "JAMAICA": "JAMAICA", "JAPAN": "JAPAN", "JORDAN": "JORDAN", "KAZAKHSTAN": "KAZAKHSTAN", "KENYA": "KENYA", "KOREA REPUBLIC OF": "SOUTH KOREA", "KUWAIT": "KUWAIT", "LEBANON": "LEBANON", "LITHUANIA": "LITHUANIA", "MACAU": "MACAU", "MALAYSIA": "MALAYSIA", "MALDIVES": "MALDIVES", "MALTA": "MALTA", "MAURITIUS": "MAURITIUS", "MEXICO": "MEXICO", "MONGOLIA": "MONGOLIA", "MONTENEGRO": "SERBIA", "MOROCCO": "MOROCCO", "NEPAL": "NEPAL", "NETHERLANDS": "NETHERLANDS", "NEW ZEALAND": "NEW ZEALAND", "NICARAGUA": "NICARAGUA", "OMAN": "OMAN", "PANAMA": "PANAMA", "PAPUA NEW GUINEA": "PAPUA NEW GUINEA", "PARAGUAY": "PARAGUAY", "PERU": "PERU", "PHILIPPINES": "PHILIPPINES", "POLAND": "POLAND", "PORTUGAL": "PORTUGAL", "PUERTO RICO": "PUERTO RICO", "QATAR": "QATAR", "ROMANIA": "ROMANIA"}
    country = CountryInfo(country_dict[country_name])
    return {f"{prefix}_country_name": country_name, f"{prefix}_continent_region": country.region(), f"{prefix}_continent": country.subregion()}






    
############################################################################################################################################################################################################################
############################################################################################################################################################################################################################
    
# Lists & Dictionary

# Holidays
from countryinfo import CountryInfo
year_list = [2020, 2021, 2022]
email_holiday_lookup_dict = {'UNITED STATES OF AMERICA': holidays.XNYS(years = year_list) + holidays.UM(years = year_list) + holidays.US(years = year_list) + holidays.VI(years = year_list),
                       'MEXICO': holidays.MX(years = year_list), 'CANADA': holidays.CA(years = year_list),
                       'LATIN AMERICA': holidays.HN(years = year_list) + holidays.NI(years = year_list) + holidays.PA(years = year_list) + holidays.AR(years = year_list) + holidays.BO(years = year_list) + holidays.BR(years = year_list) + holidays.CL(years = year_list) + holidays.CO(years = year_list) + holidays.PY(years = year_list) + holidays.PE(years = year_list) + holidays.UY(years = year_list) + holidays.VE(years = year_list),
                       'AUSTRALASIA & JAPAN': holidays.JP(years = year_list) + holidays.AS(years = year_list) + holidays.AU(years = year_list) + holidays.GU(years = year_list) + holidays.NZ(years = year_list) + holidays.MH(years = year_list) + holidays.MP(years = year_list),
                       'CARIBBEAN': holidays.AW(years = year_list) + holidays.CU(years = year_list) + holidays.CW(years = year_list) + holidays.DO(years = year_list) + holidays.JM(years = year_list) + holidays.PR(years = year_list),
                       'MAINLAND CHINA': holidays.CN(years = year_list), 'HONG KONG': holidays.HK(years = year_list), 'TAIWAN':holidays.TW(years = year_list), 'MACAU': holidays.HK(years = year_list),
                       'SOUTHEAST ASIA & KOREA': holidays.ID(years = year_list) + holidays.MY(years = year_list) + holidays.PH(years = year_list) + holidays.SG(years = year_list) + holidays.KR(years = year_list) + holidays.TH(years = year_list) + holidays.VN(years = year_list),
                       'INDIA, MIDDLE EAST & AFRICA': holidays.AO(years = year_list) + holidays.BW(years = year_list) + holidays.BI(years = year_list) + holidays.DJ(years = year_list) + holidays.EG(years = year_list) + holidays.ET(years = year_list) + holidays.KE(years = year_list) + holidays.LS(years = year_list) + holidays.MG(years = year_list) + holidays.MW(years = year_list) + holidays.MA(years = year_list) + holidays.MZ(years = year_list) + holidays.NA(years = year_list) + holidays.NG(years = year_list) + holidays.ZA(years = year_list) + holidays.TN(years = year_list) + holidays.ZM(years = year_list) + holidays.ZW(years = year_list) + holidays.CZ(years = year_list) + holidays.AM(years = year_list) + holidays.AZ(years = year_list) + holidays.BH(years = year_list) + holidays.BD(years = year_list) + holidays.GE(years = year_list) + holidays.IN(years = year_list) + holidays.IL(years = year_list) + holidays.KZ(years = year_list) + holidays.KG(years = year_list) + holidays.PK(years = year_list) + holidays.SA(years = year_list) + holidays.TR(years = year_list) + holidays.AE(years = year_list) + holidays.UZ(years = year_list),
                       'EUROPE': holidays.AL(years = year_list) + holidays.AD(years = year_list) + holidays.AT(years = year_list) + holidays.BY(years = year_list) + holidays.BE(years = year_list) + holidays.BA(years = year_list) + holidays.BG(years = year_list) + holidays.HR(years = year_list) + holidays.CY(years = year_list) + holidays.DK(years = year_list) + holidays.EE(years = year_list) + holidays.ECB(years = year_list) + holidays.FI(years = year_list) + holidays.FR(years = year_list) + holidays.DE(years = year_list) + holidays.GR(years = year_list) + holidays.HU(years = year_list) + holidays.IS(years = year_list) + holidays.IE(years = year_list) + holidays.IM(years = year_list) + holidays.IT(years = year_list) + holidays.LV(years = year_list) + holidays.LI(years = year_list) + holidays.LT(years = year_list) + holidays.LU(years = year_list) + holidays.MT(years = year_list) + holidays.MD(years = year_list) + holidays.ME(years = year_list) + holidays.NL(years = year_list) + holidays.NO(years = year_list) + holidays.PL(years = year_list) + holidays.PT(years = year_list) + holidays.RO(years = year_list) + holidays.RU(years = year_list) + holidays.SM(years = year_list) + holidays.SK(years = year_list) + holidays.SI(years = year_list) + holidays.ES(years = year_list) + holidays.SE(years = year_list) + holidays.CH(years = year_list) + holidays.UK(years = year_list) + holidays.GB(years = year_list) + holidays.SZ(years = year_list) + holidays.MK(years = year_list) + holidays.VA(years = year_list) + holidays.RS(years = year_list) + holidays.MC(years = year_list)}

year_list = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
stay_holiday_lookup_dict = {"ANDORRA": holidays.AD(years = year_list), "UNITED ARAB EMIRATES": holidays.AE(years = year_list), "ARMENIA": holidays.AM(years = year_list), "ARGENTINA": holidays.AR(years = year_list), "AUSTRIA": holidays.AT(years = year_list), "AUSTRALIA": holidays.AU(years = year_list), "ARUBA": holidays.AW(years = year_list), "AZERBAIJAN": holidays.AZ(years = year_list), "BANGLADESH": holidays.BD(years = year_list), "BELGIUM": holidays.BE(years = year_list), "BULGARIA": holidays.BG(years = year_list), "BAHRAIN": holidays.BH(years = year_list), "BRAZIL": holidays.BR(years = year_list), "BELARUS": holidays.BY(years = year_list), "CANADA": holidays.CA(years = year_list), "SWITZERLAND": holidays.CH(years = year_list), "CHILE": holidays.CL(years = year_list), "MACAU": holidays.CN(years = year_list), "MAINLAND CHINA": holidays.CN(years = year_list), "MONGOLIA": holidays.CN(years = year_list), "COLOMBIA": holidays.CO(years = year_list), "BAHAMAS": holidays.CU(years = year_list), "CYPRUS": holidays.CY(years = year_list), "CZECH REPUBLIC": holidays.CZ(years = year_list), "GERMANY": holidays.DE(years = year_list), "DENMARK": holidays.DK(years = year_list), "DOMINICAN REPUBLIC": holidays.DO(years = year_list), "EGYPT": holidays.EG(years = year_list), "SPAIN": holidays.ES(years = year_list), "FINLAND": holidays.FI(years = year_list), "FRANCE": holidays.FR(years = year_list), "UNITED KINGDOM": holidays.GB(years = year_list), "GEORGIA": holidays.GE(years = year_list), "GREECE": holidays.GR(years = year_list), "GUAM": holidays.GU(years = year_list), "HONG KONG": holidays.HK(years = year_list), "HONDURAS": holidays.HN(years = year_list), "CROATIA": holidays.HR(years = year_list), "HUNGARY": holidays.HU(years = year_list), "INDONESIA": holidays.ID(years = year_list), "IRELAND": holidays.IE(years = year_list), "ISRAEL": holidays.IL(years = year_list), "INDIA": holidays.IN(years = year_list), "MALDIVES": holidays.IN(years = year_list), "NEPAL": holidays.IN(years = year_list), "SRI LANKA": holidays.IN(years = year_list), "ICELAND": holidays.IS(years = year_list), "ITALY": holidays.IT(years = year_list), "JAMAICA": holidays.JM(years = year_list), "JAPAN": holidays.JP(years = year_list), "KENYA": holidays.KE(years = year_list), "KOREA REPUBLIC OF": holidays.KR(years = year_list), "KAZAKHSTAN": holidays.KZ(years = year_list), "LITHUANIA": holidays.LT(years = year_list), "MOROCCO": holidays.MA(years = year_list), "MONTENEGRO": holidays.ME(years = year_list), "MAURITIUS": holidays.MG(years = year_list), "SEYCHELLES": holidays.MG(years = year_list), "TANZANIA UNITED REPUBLIC OF": holidays.MG(years = year_list), "MALTA": holidays.MT(years = year_list), "MEXICO": holidays.MX(years = year_list), "MALAYSIA": holidays.MY(years = year_list), "NICARAGUA": holidays.NI(years = year_list), "NETHERLANDS": holidays.NL(years = year_list), "NEW ZEALAND": holidays.NZ(years = year_list), "PAPUA NEW GUINEA": holidays.NZ(years = year_list), "VANUATU": holidays.NZ(years = year_list), "PANAMA": holidays.PA(years = year_list), "PERU": holidays.PE(years = year_list), "PHILIPPINES": holidays.PH(years = year_list), "POLAND": holidays.PL(years = year_list), "PUERTO RICO": holidays.PR(years = year_list), "PORTUGAL": holidays.PT(years = year_list), "PARAGUAY": holidays.PY(years = year_list), "ROMANIA": holidays.RO(years = year_list), "SERBIA": holidays.RS(years = year_list), "RUSSIAN FEDERATION": holidays.RU(years = year_list), "SAUDI ARABIA": holidays.SA(years = year_list), "SWEDEN": holidays.SE(years = year_list), "SINGAPORE": holidays.SG(years = year_list), "SLOVENIA": holidays.SI(years = year_list), "SLOVAKIA": holidays.SK(years = year_list), "THAILAND": holidays.TH(years = year_list), "TURKEY": holidays.TR(years = year_list), "TAIWAN": holidays.TW(years = year_list), "UKRAINE": holidays.UA(years = year_list), "UNITED STATES": holidays.US(years = year_list), "URUGUAY": holidays.UY(years = year_list), "VENEZUELA BOLIVARIAN REPUBLIC OF": holidays.VE(years = year_list), "VIET NAM": holidays.VN(years = year_list), "TRINIDAD AND TOBAGO": holidays.VN(years = year_list), "SOUTH AFRICA": holidays.ZA(years = year_list), "ZAMBIA": holidays.ZM(years = year_list), "ZIMBABWE": holidays.ZW(years = year_list), "ALGERIA": holidays.MA(years = year_list), "CAYMAN ISLANDS": holidays.CU(years = year_list), "COSTA RICA": holidays.NI(years = year_list), "ECUADOR": holidays.PE(years = year_list), "EL SALVADOR": holidays.NI(years = year_list), "FIJI": holidays.NZ(years = year_list), "FRENCH POLYNESIA": holidays.NZ(years = year_list), "GHANA": holidays.MA(years = year_list), "GIBRALTAR": holidays.MA(years = year_list), "GUATEMALA": holidays.NI(years = year_list), "JORDAN": holidays.SA(years = year_list), "KUWAIT": holidays.SA(years = year_list), "LAO PEOPLES DEMOCRATIC REPUBLIC": holidays.TH(years = year_list), "LEBANON": holidays.SA(years = year_list), "MACEDONIA THE FORMER YUGOSLAV REPUBLIC OF": holidays.GR(years = year_list), "OMAN": holidays.SA(years = year_list), "QATAR": holidays.SA(years = year_list)}


# Subcategories
column_dict = {'CAMPAIGN_NM': {"CC_1": 0, "CC_2": 1, "CC_3": 2, "CC_4": 3},
               'MBR_TIER': {"TIER_1": 0, "TIER_2": 1, "TIER_3": 2, "TIER_4": 3},
               'MBR_PRGM_ACTV': {"LFC_2": 0, "LFC_3": 1},
               'ENROLL_CHANNEL': {'EN_0': 0, 'EN_1': 1, 'EN_2': 2, 'EN_3': 3, 'EN_4': 4, 'EN_6': 5, 'ENROLL_CHANNEL': None},
               'AGE_CD': {'AGE_0': 0, 'AGE_1': 1, 'AGE_2': 2, 'AGE_3': 3, 'AGE_4': 4, 'AGE_5': 5, 'AGE_CD': None},
               'INCOME_CD': {'INCOME_0': 0, 'INCOME_1': 1, 'INCOME_2': 2, 'INCOME_3': 3, 'INCOME_4': 4, 'INCOME_5': 5, 'INCOME_CD': None},
               'GENDER_CD': {'GNDR_0': 0, 'GNDR_1': 1, 'GNDR_2': 2, 'GENDER_CD': None},
               "BUS_LEIS_IND": {"BL_1": 0, "BL_2": 1},
               "REWARD_NT": {"RN_0": 0, "RN_1": 1, "RN_2": 2},
               "HTL_RGN": {"AMER": 0, "EMEAA": 1, "GCHINA": 2},
               "HTL_CHAIN_CATEGORY": {"CHN_CAT_0": 0, "CHN_CAT_1": 1, "CHN_CAT_2": 2, "CHN_CAT_3": 3}, 
               "HTL_CHAIN": {"CHN_1": 0, "CHN_2": 1, "CHN_3": 2, "CHN_4": 3, "CHN_5": 4, "CHN_6": 5, "CHN_7": 6, "CHN_9": 7, "CHN_10": 8, "CHN_11": 9, "CHN_12": 10, "CHN_13": 11, "CHN_14": 12, "CHN_15": 13, "CHN_18": 14, "CHN_20": 15, "CHN_22": 16, "CHN_23": 17},
               "DIFFERENT_CONTINENT": {"Same_Continent": 0, "Different_Continent": 1}
              }

# Season the email was sent
seasons = {'spring': range(3, 6), 'summer': range(6, 9), 'fall': range(9, 12), 'winter': [12, 1, 2]}

# Calculate additional columns based on rolling windows
time_windows = [7, 14, 30, 60, 90, 180, 360]

# Columns to Drop from the Members Frame during the Initial Process
init_drop_mem_cols = ["INCOME_GROUP", "MBR_REGION", "STATE_NM", "CITY_NM"]

############################################################################################################################################################################################################################
############################################################################################################################################################################################################################

# Parameters

## Email Data Initial Process
cut_date = datetime.datetime(2020, 12, 31)

## Merge Parameters
chunksize = 10000000 # 10,000,000
npartitions = 140



/gpfs/user_home/os_home_dirs/njones71/.local/lib/python3.7/site-packages/holidays/countries/marshall_islands.py:62: Warning: Years before 2022 are not available for the Marshall Islands (MH).
  Warning,


########################################################################################################################################################################

########################################################################################################################################################################

# Member Data Initial Process
1. Import member data
2. Remove Duplicated Column Names within the rows
3. Drop City and State Name Columns
4. Create Member Subregion Continent Variable
5. Translate Categorical Columns from 'Type == String' to 'Type == int8'
6. Save Member frame

In [2]:
%%time

print(f"   {datetime.datetime.now()} -- Process Started")
print(f"0. {datetime.datetime.now()} -- Import member data")
mem_dtypes = {'HASH_NBR': 'str', 'ENROLL_DT': 'str', 'ENROLL_CHANNEL': 'category', 'MBR_SUBREGION': 'category', 'STATE_NM': 'str', 'CITY_NM': 'str', 'AGE_CD': 'category', 'INCOME_CD': 'category', 'GENDER_CD': 'category'}
member_df = pd.read_csv(members_pth, dtype = mem_dtypes).drop(init_drop_mem_cols, axis = 1)

print(f"1. {datetime.datetime.now()} -- Remove Duplicated Column Names within the Rows")
member_df = member_df[member_df["HASH_NBR"] != "HASH_NBR"]

print(f"2. {datetime.datetime.now()} -- Create Member Subregion Continent")
mbr_subregion_dict = {'AUSTRALASIA & JAPAN': "Oceania", 'CANADA': "Americas", 'CARIBBEAN': "Americas", 'EUROPE': "Europe", 'HONG KONG': "Asia & Africa", 'INDIA, MIDDLE EAST & AFRICA': "Asia & Africa", 'LATIN AMERICA': "Americas", 'MAINLAND CHINA': "Asia & Africa", 'MEXICO': "Americas", 'SOUTHEAST ASIA & KOREA': "Asia & Africa", 'TAIWAN': "Asia & Africa", 'UNITED STATES OF AMERICA': "Americas", 'MACAU': "Asia & Africa", "MBR_SUBREGION": None}
member_df["MBR_continent_region"] = member_df["MBR_SUBREGION"].map(lambda row: mbr_subregion_dict[row]).astype('category')

print(f"3. {datetime.datetime.now()} -- Translate Categorical Columns from 'Type == String' to 'Type == int8'")
member_df["ENROLL_CHANNEL"] = member_df["ENROLL_CHANNEL"].map(lambda row: translate_element(row, column_dict["ENROLL_CHANNEL"])).astype('int8')
member_df["AGE_CD"] = member_df["AGE_CD"].map(lambda row: translate_element(row, column_dict["AGE_CD"])).astype('int8')
member_df["INCOME_CD"] = member_df["INCOME_CD"].map(lambda row: translate_element(row, column_dict["INCOME_CD"])).astype('int8')
member_df["GENDER_CD"] = member_df["GENDER_CD"].map(lambda row: translate_element(row, column_dict["GENDER_CD"])).astype('int8')

print(f"4. {datetime.datetime.now()} -- Save Member frame")
member_df.to_csv(members_pth_process, index = False)

print(f" N. {datetime.datetime.now()} -- Process Complete")
print(member_df.shape)
member_df.info()


   2023-04-02 00:35:55.249451 -- Process Started
0. 2023-04-02 00:35:55.249535 -- Import member data
1. 2023-04-02 00:35:57.237605 -- Remove Duplicated Column Names within the Rows
2. 2023-04-02 00:35:57.435804 -- Create Member Subregion Continent
3. 2023-04-02 00:35:57.574180 -- Translate Categorical Columns from 'Type == String' to 'Type == int8'
4. 2023-04-02 00:35:57.618851 -- Save Member frame
 N. 2023-04-02 00:36:00.473964 -- Process Complete
(1233429, 8)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233429 entries, 0 to 1233429
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   HASH_NBR              1233429 non-null  object  
 1   ENROLL_DT             1233429 non-null  object  
 2   ENROLL_CHANNEL        1233429 non-null  int8    
 3   MBR_SUBREGION         1233429 non-null  category
 4   AGE_CD                1233429 non-null  int8    
 5   INCOME_CD             1233429 non-null 

########################################################################################################################################################################

########################################################################################################################################################################

# Stay History Data Initial Process
0. Import Stay Data
1. Drop Duplicated Column Names found in Rows
2. Exchange Column Types
3. Fill in Missing Confirmation Dates with CheckIn Dates
4. Drop The Number of Nights
5. Map Hotel Rooms to Hotel Stays
6. Compute Time Between Dates, Total Rooms, Total Guest Quantity, and Average Guest Quantity per Room
7. Drop List Column ('GUEST_QTY')
8. Removing Punctuation from Hotel Country Name
9. Translate Categorical Columns via Dictionary
10. Pull Nearest Holiday for Each Stay Confirmation Date, Check-In Date, and Check-Out Date
11. Pull Country Info for each Unique Country Name
12. Unify Asia and Africa to Match the Column in the Member Dataframe
13. Merge the Member Information into the Hotel Stay Dataframe
14. Create Indicator representing whether the Hotel Stay was on a Different Conitnent than the Enrollment Continent
15. Create Member Tenure as of Stay Dates
16. Drop Columns No Longer Needed
17. Create Columns of Placeholder Zeros
18. Dummify Categorical Columns
19. Drop Original Columns
20. Save Frame

In [5]:
%%time

print(f"    {datetime.datetime.now()} -- Process Started")
print(f" 0. {datetime.datetime.now()} -- Import Stay Data")
stay_dtype = {'HASH_NBR': "str", 'CONF_HASH_NBR': "str", 'CONF_DT': "str", 'CK_IN_DT': "str", 'CK_OUT_DT': "str", 'HTL_HASH_NBR': "str", 'HTL_RGN': "str", 'HTL_CTRY_NM': "str", 'HTL_CITY_NM': "str", 'HTL_CHAIN': "str", 'GUEST_QTY': "str", 'REWARD_NT': "str", 'NBR_OF_NIGHTS': "str", 'ROOM_REVENUE_USD': "str", 'BUS_LEIS_IND': "str", 'HTL_CHAIN_CATEGORY': "str"}
stay_hist_df = pd.read_csv(stay_hist_pth, dtype = stay_dtype)

print(f" 1. {datetime.datetime.now()} -- Drop Duplicated Column Names found in Rows")
stay_hist_df = stay_hist_df[stay_hist_df["HASH_NBR"] != "HASH_NBR"]

print(f" 2. {datetime.datetime.now()} -- Exchange Column Types")
stay_hist_df["ROOM_REVENUE_USD"] = stay_hist_df["ROOM_REVENUE_USD"].astype('float64')
stay_hist_df["GUEST_QTY"] = stay_hist_df["GUEST_QTY"].fillna("0").astype('int16')
stay_hist_df["CONF_DT"] = pd.to_datetime(stay_hist_df["CONF_DT"])
stay_hist_df["CK_IN_DT"] = pd.to_datetime(stay_hist_df["CK_IN_DT"])
stay_hist_df["CK_OUT_DT"] = pd.to_datetime(stay_hist_df["CK_OUT_DT"])

print(f" 3. {datetime.datetime.now()} -- Fill in Missing Confirmation Dates with CheckIn Dates")
stay_hist_df.loc[stay_hist_df["CONF_DT"].isna(), "CONF_DT"] = stay_hist_df.loc[stay_hist_df["CONF_DT"].isna(), "CK_IN_DT"]

print(f" 4. {datetime.datetime.now()} -- Drop The Number of Nights")
stay_hist_df.drop(["NBR_OF_NIGHTS", "HTL_CITY_NM"], axis = 1, inplace = True)

print(f" 5. {datetime.datetime.now()} -- Map Hotel Rooms to Hotel Stays")
stay_hist_df = stay_hist_df.groupby(["CONF_HASH_NBR",
                                     "HASH_NBR",
                                     "BUS_LEIS_IND",
                                     "REWARD_NT",
                                     "CONF_DT",
                                     "CK_IN_DT",
                                     "CK_OUT_DT", 
                                     "HTL_HASH_NBR",
                                     "HTL_RGN",
                                     "HTL_CTRY_NM",
                                     "HTL_CHAIN",
                                     "HTL_CHAIN_CATEGORY"
                                    ]).agg({"GUEST_QTY": list, "ROOM_REVENUE_USD": sum}).rename(columns = {"ROOM_REVENUE_USD": "STAY_REVENUE_USD"}).reset_index()

print(f" 6. {datetime.datetime.now()} -- Compute Time Between Dates, Total Rooms, Total Guest Quantity, and Average Guest Quantity per Room")
stay_hist_df["total_rooms"] = [len(x) for x in stay_hist_df["GUEST_QTY"]]
stay_hist_df["total_GUEST_QTY"] = [np.sum(x) for x in stay_hist_df["GUEST_QTY"]]
stay_hist_df.loc[:, ["total_rooms", "total_GUEST_QTY"]] = stay_hist_df.loc[:, ["total_rooms", "total_GUEST_QTY"]].astype('int16')
stay_hist_df["TIME_BWTN_CONF_CKIN_DT"] = (stay_hist_df["CK_IN_DT"] - stay_hist_df["CONF_DT"]).dt.days.astype('int16')
stay_hist_df["TIME_BWTN_CONF_CKOUT_DT"] = (stay_hist_df["CK_OUT_DT"] - stay_hist_df["CONF_DT"]).dt.days.astype('int16')
stay_hist_df["TIME_BWTN_CKIN_CKOUT_DT"] = (stay_hist_df["CK_OUT_DT"] - stay_hist_df["CK_IN_DT"]).dt.days.astype('int16')

print(f" 7. {datetime.datetime.now()} -- Drop List Column ('GUEST_QTY')")
stay_hist_df.drop("GUEST_QTY", axis = 1, inplace = True)

print(f" 8. {datetime.datetime.now()} -- Removing Punctuation from Hotel Country Name")
stay_hist_df["HTL_CTRY_NM"] = [re.sub("[^A-Za-z0-9 ]+", "", x) for x in stay_hist_df["HTL_CTRY_NM"]]

print(f" 9. {datetime.datetime.now()} -- Translate Categorical Columns via Dictionary")
stay_hist_df["BUS_LEIS_IND"] = stay_hist_df["BUS_LEIS_IND"].map(lambda row: translate_element(row, column_dict["BUS_LEIS_IND"])).astype('int8')
stay_hist_df["REWARD_NT"] = stay_hist_df["REWARD_NT"].map(lambda row: translate_element(row, column_dict["REWARD_NT"])).astype('int8')
stay_hist_df["HTL_CHAIN"] = stay_hist_df["HTL_CHAIN"].map(lambda row: translate_element(row, column_dict["HTL_CHAIN"])).astype('int8')
stay_hist_df["HTL_CHAIN_CATEGORY"] = stay_hist_df["HTL_CHAIN_CATEGORY"].map(lambda row: translate_element(row, column_dict["HTL_CHAIN_CATEGORY"])).astype('int8')
stay_hist_df["HTL_RGN"] = stay_hist_df["HTL_RGN"].map(lambda row: translate_element(row, column_dict["HTL_RGN"])).astype('int8')

print(f"10. {datetime.datetime.now()} -- Pull Nearest Holiday for Each Stay Confirmation Date, Check-In Date, and Check-Out Date")
stay_hist_df.loc[:, ["CONF_DT_daysto_holiday", "CONF_DT_nearest_holiday_date", "CONF_DT_nearest_holiday_name", "CK_IN_DT_daysto_holiday", "CK_IN_DT_nearest_holiday_date", "CK_IN_DT_nearest_holiday_name", "CK_OUT_DT_daysto_holiday", "CK_OUT_DT_nearest_holiday_date", "CK_OUT_DT_nearest_holiday_name"]] = [get_nearest_holiday([conf_date, ckin_date, ckout_date], subregion, stay_holiday_lookup_dict) for conf_date, ckin_date, ckout_date, subregion in zip(stay_hist_df["CONF_DT"], stay_hist_df["CK_IN_DT"], stay_hist_df["CK_OUT_DT"], stay_hist_df["HTL_CTRY_NM"])]
stay_hist_df.loc[:, ["CONF_DT_daysto_holiday", "CK_IN_DT_daysto_holiday", "CK_OUT_DT_daysto_holiday"]] = stay_hist_df.loc[:, ["CONF_DT_daysto_holiday", "CK_IN_DT_daysto_holiday", "CK_OUT_DT_daysto_holiday"]].astype('int16')

stay_hist_df["CONF_DT_nearest_holiday_date"] = pd.to_datetime(stay_hist_df["CONF_DT_nearest_holiday_date"])
stay_hist_df["CK_IN_DT_nearest_holiday_date"] = pd.to_datetime(stay_hist_df["CK_IN_DT_nearest_holiday_date"])
stay_hist_df["CK_OUT_DT_nearest_holiday_date"] = pd.to_datetime(stay_hist_df["CK_OUT_DT_nearest_holiday_date"])
stay_hist_df['CONF_DT_nearest_holiday_dayofyear'] = stay_hist_df['CONF_DT_nearest_holiday_date'].dt.dayofyear
stay_hist_df['CK_IN_DT_nearest_holiday_dayofyear'] = stay_hist_df['CK_IN_DT_nearest_holiday_date'].dt.dayofyear
stay_hist_df['CK_OUT_DT_nearest_holiday_dayofyear'] = stay_hist_df['CK_OUT_DT_nearest_holiday_date'].dt.dayofyear

stay_hist_df.loc[:, ["CONF_DT_nearest_holiday_dayofyear", "CK_IN_DT_nearest_holiday_dayofyear", "CK_OUT_DT_nearest_holiday_dayofyear"]] = stay_hist_df.loc[:, ["CONF_DT_nearest_holiday_dayofyear", "CK_IN_DT_nearest_holiday_dayofyear", "CK_OUT_DT_nearest_holiday_dayofyear"]].astype('int16')

print(f"11. {datetime.datetime.now()} -- Pull Country Info for each Unique Country Name")
stay_hist_df = pd.merge(left = stay_hist_df,
                        right = pd.DataFrame([get_country_info(country_name, "HTL_CTRY") for country_name in list(set(stay_hist_df["HTL_CTRY_NM"].to_list()))]),
                        left_on = "HTL_CTRY_NM",
                        right_on = "HTL_CTRY_country_name",
                        how = "left")

print(f"12. {datetime.datetime.now()} -- Unify Asia and Africa to Match the Column in the Member Dataframe")
temp_dict = {"Africa": "Asia & Africa", "Asia": "Asia & Africa", "Americas": "Americas", "Europe": "Europe", "Oceania": "Oceania", "Asia & Africa": "Asia & Africa", np.nan: np.nan}
stay_hist_df["HTL_CTRY_continent_region"] = [temp_dict[x] for x in stay_hist_df["HTL_CTRY_continent_region"]]

print(f"13. {datetime.datetime.now()} -- Merge the Member Information into the Hotel Stay Dataframe")
stay_hist_df = pd.merge(left = stay_hist_df,
                        right = member_df.loc[:, ["HASH_NBR", "ENROLL_DT", "MBR_continent_region"]],
                        left_on = "HASH_NBR",
                        right_on = "HASH_NBR",
                        how = "left")

print(f"14. {datetime.datetime.now()} -- Create Indicator representing whether the Hotel Stay was on a Different Conitnent than the Enrollment Continent")
stay_hist_df["DIFFERENT_CONTINENT"] = ["Different_Continent" if htl_cont != mbr_cont else "Same_Continent" for htl_cont, mbr_cont in zip(stay_hist_df["HTL_CTRY_continent_region"], stay_hist_df["MBR_continent_region"])]
stay_hist_df["DIFFERENT_CONTINENT"] = stay_hist_df["DIFFERENT_CONTINENT"].map(lambda row: translate_element(row, column_dict["DIFFERENT_CONTINENT"])).astype('int8')
 
print(f"15. {datetime.datetime.now()} -- Create Member Tenure as of Stay Dates")
stay_hist_df["ENROLL_DT"] = pd.to_datetime(stay_hist_df["ENROLL_DT"])
stay_hist_df["MBR_TENURE_ASOF_CONF_DT"] = (stay_hist_df["CONF_DT"] - stay_hist_df["ENROLL_DT"]).dt.days.astype('int32')
stay_hist_df["MBR_TENURE_ASOF_CK_IN_DT"] = (stay_hist_df["CK_IN_DT"] - stay_hist_df["ENROLL_DT"]).dt.days.astype('int32')
stay_hist_df["MBR_TENURE_ASOF_CK_OUT_DT"] = (stay_hist_df["CK_OUT_DT"] - stay_hist_df["ENROLL_DT"]).dt.days.astype('int32')

stay_hist_df.loc[:, ["MBR_TENURE_ASOF_CONF_DT", "MBR_TENURE_ASOF_CK_IN_DT", "MBR_TENURE_ASOF_CK_OUT_DT"]] = stay_hist_df.loc[:, ["MBR_TENURE_ASOF_CONF_DT", "MBR_TENURE_ASOF_CK_IN_DT", "MBR_TENURE_ASOF_CK_OUT_DT"]].astype('int16')

print(f"16. {datetime.datetime.now()} -- Drop Columns No Longer Needed")
stay_hist_df.drop([# This Variable will be represented by the sum of TIME_BWTN_CONF_CKIN_DT + TIME_BWTN_CKIN_CKOUT_DT
                   'TIME_BWTN_CONF_CKOUT_DT', 
                   # These Variables will be represented numerically by *_nearest_holiday_dayofyear
                   'CONF_DT_nearest_holiday_date',
                   'CK_IN_DT_nearest_holiday_date', 
                   'CK_OUT_DT_nearest_holiday_date',
                   'CONF_DT_nearest_holiday_name', 
                   'CK_IN_DT_nearest_holiday_name',
                   'CK_OUT_DT_nearest_holiday_name', 
                   # These Variables were used to create DIFFERENT_CONTINENT
                   'MBR_continent_region',
                   'HTL_CTRY_continent_region',
                   'HTL_CTRY_continent', 
                   'HTL_CTRY_NM',
                   # These Variables have too many categories and is highly related to DIFFERENT_CONTINENT
                   'HTL_CTRY_country_name', 
                   # This Variable was used to create Member Tenure as of Hotel Stay Dates
                   'ENROLL_DT'
                  ], 
                  axis = 1,
                  inplace = True  
                 )

print(f"17. {datetime.datetime.now()} -- Create Columns of Placeholder Zeros")
stay_hist_df["HTL_RGN_EMEAA"] = 0
stay_hist_df["HTL_RGN_GCHINA"] = 0
stay_hist_df["HTL_CHAIN_CATEGORY_CHN_CAT_1"] = 0
stay_hist_df["HTL_CHAIN_CATEGORY_CHN_CAT_2"] = 0
stay_hist_df["HTL_CHAIN_CATEGORY_CHN_CAT_3"] = 0
stay_hist_df["REWARD_NT_RN_1"] = 0
stay_hist_df["REWARD_NT_RN_2"] = 0
stay_hist_df["HTL_CHAIN_CHN_2"] = 0
stay_hist_df["HTL_CHAIN_CHN_3"] = 0
stay_hist_df["HTL_CHAIN_CHN_4"] = 0
stay_hist_df["HTL_CHAIN_CHN_5"] = 0
stay_hist_df["HTL_CHAIN_CHN_6"] = 0
stay_hist_df["HTL_CHAIN_CHN_7"] = 0
stay_hist_df["HTL_CHAIN_CHN_9"] = 0
stay_hist_df["HTL_CHAIN_CHN_10"] = 0
stay_hist_df["HTL_CHAIN_CHN_11"] = 0
stay_hist_df["HTL_CHAIN_CHN_12"] = 0
stay_hist_df["HTL_CHAIN_CHN_13"] = 0
stay_hist_df["HTL_CHAIN_CHN_14"] = 0
stay_hist_df["HTL_CHAIN_CHN_15"] = 0
stay_hist_df["HTL_CHAIN_CHN_18"] = 0
stay_hist_df["HTL_CHAIN_CHN_20"] = 0
stay_hist_df["HTL_CHAIN_CHN_22"] = 0
stay_hist_df["HTL_CHAIN_CHN_23"] = 0

print(f"18. {datetime.datetime.now()} -- Dummify Categorical Columns")
stay_hist_df.loc[stay_hist_df["HTL_RGN"] == 1, "HTL_RGN_EMEAA"] = 1
stay_hist_df.loc[stay_hist_df["HTL_RGN"] == 2, "HTL_RGN_GCHINA"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN_CATEGORY"] == 1, "HTL_CHAIN_CATEGORY_CHN_CAT_1"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN_CATEGORY"] == 2, "HTL_CHAIN_CATEGORY_CHN_CAT_2"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN_CATEGORY"] == 3, "HTL_CHAIN_CATEGORY_CHN_CAT_3"] = 1
stay_hist_df.loc[stay_hist_df["REWARD_NT"] == 1, "REWARD_NT_RN_1"] = 1
stay_hist_df.loc[stay_hist_df["REWARD_NT"] == 2, "REWARD_NT_RN_2"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 1, "HTL_CHAIN_CHN_2"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 2, "HTL_CHAIN_CHN_3"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 3, "HTL_CHAIN_CHN_4"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 4, "HTL_CHAIN_CHN_5"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 5, "HTL_CHAIN_CHN_6"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 6, "HTL_CHAIN_CHN_7"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 7, "HTL_CHAIN_CHN_9"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 8, "HTL_CHAIN_CHN_10"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 9, "HTL_CHAIN_CHN_11"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 10, "HTL_CHAIN_CHN_12"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 11, "HTL_CHAIN_CHN_13"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 12, "HTL_CHAIN_CHN_14"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 13, "HTL_CHAIN_CHN_15"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 14, "HTL_CHAIN_CHN_18"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 15, "HTL_CHAIN_CHN_20"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 16, "HTL_CHAIN_CHN_22"] = 1
stay_hist_df.loc[stay_hist_df["HTL_CHAIN"] == 17, "HTL_CHAIN_CHN_23"] = 1

stay_hist_df.loc[:, ['HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']] = stay_hist_df.loc[:, ['HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']].astype('int8')

print(f"19. {datetime.datetime.now()} -- Drop Original Columns")
stay_hist_df.drop(["HTL_RGN", "HTL_CHAIN", "HTL_CHAIN_CATEGORY", "REWARD_NT"], axis = 1, inplace = True)

print(f"20. {datetime.datetime.now()} -- Save Frame")
stay_hist_df.to_csv(stay_hist_pth_process, index = False)

print(f" N. {datetime.datetime.now()} -- Process Complete")
print(stay_hist_df.shape)
stay_hist_df.info()


16. 2023-04-01 20:38:10.239683 -- Drop Columns No Longer Needed
17. 2023-04-01 20:38:11.410337 -- Create Columns of Placeholder Zeros
18. 2023-04-01 20:38:11.877452 -- Dummify Categorical Columns
19. 2023-04-01 20:38:24.800192 -- Drop Original Columns
20. 2023-04-01 20:38:26.131823 -- Save Frame
 N. 2023-04-01 20:40:18.473333 -- Process Complete
(4708642, 46)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4708642 entries, 0 to 4708641
Data columns (total 46 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   CONF_HASH_NBR                        object        
 1   HASH_NBR                             object        
 2   BUS_LEIS_IND                         int8          
 3   CONF_DT                              datetime64[ns]
 4   CK_IN_DT                             datetime64[ns]
 5   CK_OUT_DT                            datetime64[ns]
 6   HTL_HASH_NBR                         object        
 7   STAY_RE

########################################################################################################################################################################

########################################################################################################################################################################

# Email History Data Initial Process
1. Remove duplicated column names from the rows
2. Convert the Email Send Date from 'Type == String' to 'Type == Datetime'
3. Remove data prior to 1/1/2021
4. Convert categorical columns ['CAMPAIGN_NM', 'MBR_TIER', 'MBR_PRGM_ACTV'] from 'Type == String' to 'Type == int8'
5. Create Time relative features
6. Merge the Member's Information into the Current Chunk
7. Append frame to email_hist_list
8. Join frames within 'email_hist_list' into one frame
9. Save frame to a csv.

In [ ]:
%%time

print(f"{datetime.datetime.now()} -- Process Started")
email_hist_list = []
em_type = {"HASH_NBR": "str", "CAMPAIGN_NBR": "str", "UNSUB_IND": "category", "SEND_DT": "str", "CLICK": "category", "CAMPAIGN_NM": "category", "MBR_TIER": "category", "MBR_PRGM_ACTV": "category"}
for i, email_hist_df in enumerate(pd.read_csv(email_hist_pth, chunksize = chunksize, dtype = em_type)):
    print(f"Iteration: {i} -- {datetime.datetime.now()}")
    email_hist_df = email_hist_df.loc[email_hist_df["HASH_NBR"] != "HASH_NBR"].copy()
    email_hist_df["SEND_DT"] = pd.to_datetime(email_hist_df["SEND_DT"])
    email_hist_df["CAMPAIGN_NM"] = email_hist_df["CAMPAIGN_NM"].map(lambda row: translate_element(row, column_dict["CAMPAIGN_NM"])).astype('int8')
    email_hist_df["MBR_TIER"] = email_hist_df["MBR_TIER"].map(lambda row: translate_element(row, column_dict["MBR_TIER"])).astype('int8')
    email_hist_df["MBR_PRGM_ACTV"] = email_hist_df["MBR_PRGM_ACTV"].map(lambda row: translate_element(row, column_dict["MBR_PRGM_ACTV"])).astype('int8')
    email_hist_df['quarter'] = email_hist_df['SEND_DT'].dt.quarter.astype('int8')
    email_hist_df['week_of_year'] = email_hist_df['SEND_DT'].dt.strftime("%W").astype('int8')
    email_hist_df['day_of_week'] = email_hist_df['SEND_DT'].dt.dayofweek.astype('int8')
    email_hist_df['day_of_month'] = email_hist_df['SEND_DT'].dt.day.astype('int8')
    email_hist_df['day_of_year'] = email_hist_df['SEND_DT'].dt.dayofyear.astype('int16')
    email_hist_df['month'] = email_hist_df['SEND_DT'].dt.month.astype('int8')
    email_hist_df['is_end_month'] = email_hist_df['SEND_DT'].dt.is_month_end.astype('int8')
    email_hist_df['is_start_month'] = email_hist_df['SEND_DT'].dt.is_month_start.astype('int8')
    email_hist_df['is_end_quarter'] = email_hist_df['SEND_DT'].dt.is_quarter_end.astype('int8')
    email_hist_df['is_start_quarter'] = email_hist_df['SEND_DT'].dt.is_quarter_start.astype('int8')
    email_hist_df['season'] = email_hist_df['month'].apply(get_season).astype('category')
    email_hist_df = pd.merge(left = email_hist_df, right = member_df, left_on = "HASH_NBR", right_on = "HASH_NBR", how = "left")
    email_hist_df["MBR_TENURE"] = (email_hist_df["SEND_DT"] - email_hist_df["ENROLL_DT"]).dt.day.fillna(0).astype('int32')
    email_hist_df["CC_2"] = 0   
    email_hist_df["CC_3"] = 0   
    email_hist_df["CC_4"] = 0   
    email_hist_df["TIER_2"] = 0   
    email_hist_df["TIER_3"] = 0   
    email_hist_df["TIER_4"] = 0   
    email_hist_df["quarter_1"] = 0   
    email_hist_df["quarter_3"] = 0   
    email_hist_df["quarter_4"] = 0   
    email_hist_df["GNDR_1"] = 0    
    email_hist_df["GNDR_2"] = 0    
    email_hist_df["EN_1"] = 0
    email_hist_df["EN_2"] = 0
    email_hist_df["EN_3"] = 0
    email_hist_df["EN_4"] = 0
    email_hist_df["EN_6"] = 0
    email_hist_df["AGE_1"] = 0
    email_hist_df["AGE_2"] = 0
    email_hist_df["AGE_3"] = 0
    email_hist_df["AGE_4"] = 0
    email_hist_df["AGE_5"] = 0
    email_hist_df["INCOME_1"] = 0
    email_hist_df["INCOME_2"] = 0
    email_hist_df["INCOME_3"] = 0
    email_hist_df["INCOME_4"] = 0
    email_hist_df["INCOME_5"] = 0
    email_hist_df["January"] = 0   
    email_hist_df["February"] = 0   
    email_hist_df["April"] = 0   
    email_hist_df["May"] = 0   
    email_hist_df["June"] = 0     
    email_hist_df["July"] = 0   
    email_hist_df["August"] = 0   
    email_hist_df["September"] = 0   
    email_hist_df["October"] = 0   
    email_hist_df["November"] = 0   
    email_hist_df["December"] = 0    
    email_hist_df["Monday"] = 0   
    email_hist_df["Tuesday"] = 0   
    email_hist_df["Wednesday"] = 0   
    email_hist_df["Thursday"] = 0   
    email_hist_df["Friday"] = 0   
    email_hist_df["Saturday"] = 0     
    email_hist_df["Fall"] = 0   
    email_hist_df["Summer"] = 0   
    email_hist_df["Winter"] = 0    
    email_hist_df["Oceania"] = 0   
    email_hist_df["Asia_Africa"] = 0   
    email_hist_df["Europe"] = 0  
    email_hist_df["AUSTRALASIA_JAPAN"] = 0
    email_hist_df['CANADA'] = 0
    email_hist_df['CARIBBEAN'] = 0
    email_hist_df['EUROPE'] = 0
    email_hist_df['HONG_KONG'] = 0
    email_hist_df['INDIA_MIDDLE_EAST_AFRICA'] = 0
    email_hist_df['LATIN_AMERICA'] = 0
    email_hist_df['MAINLAND_CHINA'] = 0
    email_hist_df['MEXICO'] = 0
    email_hist_df['SOUTHEAST_ASIA_KOREA'] = 0
    email_hist_df['TAIWAN'] = 0
    email_hist_df['MACAU'] = 0
    email_hist_df.loc[email_hist_df["CAMPAIGN_NM"] == 1, "CC_2"] = 1
    email_hist_df.loc[email_hist_df["CAMPAIGN_NM"] == 2, "CC_3"] = 1
    email_hist_df.loc[email_hist_df["CAMPAIGN_NM"] == 3, "CC_4"] = 1
    email_hist_df.loc[email_hist_df["MBR_TIER"] == 1, "TIER_2"] = 1
    email_hist_df.loc[email_hist_df["MBR_TIER"] == 2, "TIER_3"] = 1
    email_hist_df.loc[email_hist_df["MBR_TIER"] == 3, "TIER_4"] = 1
    email_hist_df.loc[email_hist_df["quarter"] == 1, "quarter" + "_" + "1"] = 1
    email_hist_df.loc[email_hist_df["quarter"] == 3, "quarter" + "_" + "3"] = 1
    email_hist_df.loc[email_hist_df["quarter"] == 4, "quarter" + "_" + "4"] = 1
    email_hist_df.loc[email_hist_df["GENDER_CD"] == 1, "GNDR_1"] = 1
    email_hist_df.loc[email_hist_df["GENDER_CD"] == 2, "GNDR_2"] = 1
    email_hist_df.loc[email_hist_df["ENROLL_CHANNEL"] == 1, "EN_1"] = 1
    email_hist_df.loc[email_hist_df["ENROLL_CHANNEL"] == 2, "EN_2"] = 1
    email_hist_df.loc[email_hist_df["ENROLL_CHANNEL"] == 3, "EN_3"] = 1
    email_hist_df.loc[email_hist_df["ENROLL_CHANNEL"] == 4, "EN_4"] = 1
    email_hist_df.loc[email_hist_df["ENROLL_CHANNEL"] == 5, "EN_6"] = 1
    email_hist_df.loc[email_hist_df["AGE_CD"] == 1, "AGE_1"] = 1
    email_hist_df.loc[email_hist_df["AGE_CD"] == 2, "AGE_2"] = 1
    email_hist_df.loc[email_hist_df["AGE_CD"] == 3, "AGE_3"] = 1
    email_hist_df.loc[email_hist_df["AGE_CD"] == 4, "AGE_4"] = 1
    email_hist_df.loc[email_hist_df["AGE_CD"] == 5, "AGE_5"] = 1
    email_hist_df.loc[email_hist_df["INCOME_CD"] == 1, "INCOME_1"] = 1
    email_hist_df.loc[email_hist_df["INCOME_CD"] == 2, "INCOME_2"] = 1
    email_hist_df.loc[email_hist_df["INCOME_CD"] == 3, "INCOME_3"] = 1
    email_hist_df.loc[email_hist_df["INCOME_CD"] == 4, "INCOME_4"] = 1
    email_hist_df.loc[email_hist_df["INCOME_CD"] == 5, "INCOME_5"] = 1
    email_hist_df.loc[email_hist_df["month"] == 0, "January"] = 1
    email_hist_df.loc[email_hist_df["month"] == 1, "February"] = 1
    email_hist_df.loc[email_hist_df["month"] == 3, "April"] = 1
    email_hist_df.loc[email_hist_df["month"] == 4, "May"] = 1
    email_hist_df.loc[email_hist_df["month"] == 5, "June"] = 1
    email_hist_df.loc[email_hist_df["month"] == 0, "July"] = 1
    email_hist_df.loc[email_hist_df["month"] == 1, "August"] = 1
    email_hist_df.loc[email_hist_df["month"] == 2, "September"] = 1
    email_hist_df.loc[email_hist_df["month"] == 3, "October"] = 1
    email_hist_df.loc[email_hist_df["month"] == 4, "November"] = 1
    email_hist_df.loc[email_hist_df["month"] == 5, "December"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 0, "Monday"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 1, "Tuesday"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 2, "Wednesday"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 3, "Thursday"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 4, "Friday"] = 1
    email_hist_df.loc[email_hist_df["day_of_week"] == 5, "Saturday"] = 1
    email_hist_df.loc[email_hist_df["season"] == 'Fall', "Fall"] = 1
    email_hist_df.loc[email_hist_df["season"] == 'Summer', "Summer"] = 1
    email_hist_df.loc[email_hist_df["season"] == 'Winter', "Winter"] = 1
    email_hist_df.loc[email_hist_df["MBR_continent_region"] == 'Oceania', "Oceania"] = 1
    email_hist_df.loc[email_hist_df["MBR_continent_region"] == 'Asia & Africa', "Asia_Africa"] = 1
    email_hist_df.loc[email_hist_df["MBR_continent_region"] == 'Europe', "Europe"] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'AUSTRALASIA & JAPAN', 'AUSTRALASIA_JAPAN'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'CANADA', 'CANADA'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'CARIBBEAN', 'CARIBBEAN'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'EUROPE', 'EUROPE'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'HONG KONG', 'HONG_KONG'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'INDIA, MIDDLE EAST & AFRICA', 'INDIA_MIDDLE_EAST_AFRICA'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'LATIN AMERICA', 'LATIN_AMERICA'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'MAINLAND CHINA', 'MAINLAND_CHINA'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'MEXICO', 'MEXICO'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'SOUTHEAST ASIA & KOREA', 'SOUTHEAST_ASIA_KOREA'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'TAIWAN', 'TAIWAN'] = 1
    email_hist_df.loc[email_hist_df["MBR_SUBREGION"] == 'MACAU', 'MACAU'] = 1
    email_hist_df.loc[:, ["CC_2", "CC_3", "CC_4", "TIER_2", "TIER_3", "TIER_4", "quarter_1", "quarter_3", "quarter_4", "GNDR_1", "GNDR_2", "EN_1", "EN_2", "EN_3", "EN_4", "EN_6", "AGE_1", "AGE_2", "AGE_3", "AGE_4", "AGE_5", "INCOME_1", "INCOME_2", "INCOME_3", "INCOME_4", "INCOME_5", "January", "February", "April", "May", "June", "July", "August", "September", "October", "November", "December", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Fall", "Summer", "Winter", "Oceania", "Asia_Africa", "Europe", "AUSTRALASIA_JAPAN", "CANADA", "CARIBBEAN", "EUROPE", "HONG_KONG", "INDIA_MIDDLE_EAST_AFRICA", "LATIN_AMERICA", "MAINLAND_CHINA", "MEXICO", "SOUTHEAST_ASIA_KOREA", "TAIWAN", "MACAU"]] = email_hist_df.loc[:, ["CC_2", "CC_3", "CC_4", "TIER_2", "TIER_3", "TIER_4", "quarter_1", "quarter_3", "quarter_4", "GNDR_1", "GNDR_2", "EN_1", "EN_2", "EN_3", "EN_4", "EN_6", "AGE_1", "AGE_2", "AGE_3", "AGE_4", "AGE_5", "INCOME_1", "INCOME_2", "INCOME_3", "INCOME_4", "INCOME_5", "January", "February", "April", "May", "June", "July", "August", "September", "October", "November", "December", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Fall", "Summer", "Winter", "Oceania", "Asia_Africa", "Europe", "AUSTRALASIA_JAPAN", "CANADA", "CARIBBEAN", "EUROPE", "HONG_KONG", "INDIA_MIDDLE_EAST_AFRICA", "LATIN_AMERICA", "MAINLAND_CHINA", "MEXICO", "SOUTHEAST_ASIA_KOREA", "TAIWAN", "MACAU"]].astype('int8')
    email_hist_df.drop(["INCOME_CD", "AGE_CD", "ENROLL_CHANNEL", "GENDER_CD", "quarter", "MBR_TIER", "CAMPAIGN_NM", "day_of_week", "month", "season", "MBR_continent_region", "MBR_SUBREGION"], axis = 1, inplace = True)
    email_hist_list.append(email_hist_df)
    
print(f"{datetime.datetime.now()} -- Save Results")
email_hist_list[0].to_csv(email_hist_pth_process, index=False)

for df in email_hist_list[1:]:
    df.to_csv(email_hist_pth_process, mode='a', header=False, index=False)

print(f"{datetime.datetime.now()} -- Process Complete")
print(email_hist_df.shape)
email_hist_df.dtypes
    

########################################################################################################################################################################

########################################################################################################################################################################

# Email and Hotel Stay Data Merge Process

In [ ]:
em_proc_dict = {'HASH_NBR': "object", 'CAMPAIGN_NBR': "object", 'UNSUB_IND': "int8", 'SEND_DT': "object", 'CLICK': "int8", 'MBR_PRGM_ACTV': "int8", 'week_of_year': "int8", 'day_of_month': "int8", 'day_of_year': "int8", 'is_end_month': "int8", 'is_start_month': "int8", 'is_end_quarter': "int8", 'is_start_quarter': "int8", 'ENROLL_DT': "object", 'CC_2': "int8", 'CC_3': "int8", 'CC_4': "int8", 'TIER_2': "int8", 'TIER_3': "int8", 'TIER_4': "int8", 'quarter_1': "int8", 'quarter_3': "int8", 'quarter_4': "int8", 'GNDR_1': "int8", 'GNDR_2': "int8", 'EN_1': "int8", 'EN_2': "int8", 'EN_3': "int8", 'EN_4': "int8", 'EN_6': "int8", 'AGE_1': "int8", 'AGE_2': "int8", 'AGE_3': "int8", 'AGE_4': "int8", 'AGE_5': "int8", 'INCOME_1': "int8", 'INCOME_2': "int8", 'INCOME_3': "int8", 'INCOME_4': "int8", 'INCOME_5': "int8", 'January': "int8", 'February': "int8", 'April': "int8", 'May': "int8", 'June': "int8", 'July': "int8", 'August': "int8", 'September': "int8", 'October': "int8", 'November': "int8", 'December': "int8", 'Monday': "int8", 'Tuesday': "int8", 'Wednesday': "int8", 'Thursday': "int8", 'Friday': "int8", 'Saturday': "int8", 'Fall': "int8", 'Summer': "int8", 'Winter': "int8", 'Oceania': "int8", 'Asia_Africa': "int8", 'Europe': "int8", 'AUSTRALASIA_JAPAN': "int8", 'CANADA': "int8", 'CARIBBEAN': "int8", 'EUROPE': "int8", 'HONG_KONG': "int8", 'INDIA_MIDDLE_EAST_AFRICA': "int8", 'LATIN_AMERICA': "int8", 'MAINLAND_CHINA': "int8", 'MEXICO': "int8", 'SOUTHEAST_ASIA_KOREA': "int8", 'TAIWAN': "int8", 'MACAU': "int8"}
column_name_list = ['CONF_HASH_NBR', 'CONF_DT', 'CK_IN_DT', 'HTL_HASH_NBR', 'STAY_REVENUE_USD', 'total_rooms', 'total_GUEST_QTY', 'BUS_LEIS_IND', 'TIME_BWTN_CONF_CKIN_DT', 'TIME_BWTN_CKIN_CKOUT_DT', 'CONF_DT_daysto_holiday', 'CK_IN_DT_daysto_holiday', 'CK_OUT_DT_daysto_holiday', 'CONF_DT_nearest_holiday_dayofyear', 'CK_IN_DT_nearest_holiday_dayofyear', 'CK_OUT_DT_nearest_holiday_dayofyear', 'DIFFERENT_CONTINENT', 'MBR_TENURE_ASOF_CONF_DT', 'MBR_TENURE_ASOF_CK_IN_DT', 'MBR_TENURE_ASOF_CK_OUT_DT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']
cat_col_names_list = ['BUS_LEIS_IND', 'DIFFERENT_CONTINENT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']
stay_type = {"CONF_HASH_NBR":"object", "HASH_NBR":"object", "BUS_LEIS_IND":"int8", "CONF_DT":"object", "CK_IN_DT":"object", "CK_OUT_DT":"object", "HTL_HASH_NBR":"object", "STAY_REVENUE_USD":"float64", "total_rooms":"int16", "total_GUEST_QTY":"int16", "TIME_BWTN_CONF_CKIN_DT":"int16", "TIME_BWTN_CKIN_CKOUT_DT":"int16", "CONF_DT_daysto_holiday":"int16", "CK_IN_DT_daysto_holiday":"int16", "CK_OUT_DT_daysto_holiday":"int16", "CONF_DT_nearest_holiday_dayofyear":"int16", "CK_IN_DT_nearest_holiday_dayofyear":"int16", "CK_OUT_DT_nearest_holiday_dayofyear":"int16", "DIFFERENT_CONTINENT":"int8", "MBR_TENURE_ASOF_CONF_DT":"int16", "MBR_TENURE_ASOF_CK_IN_DT":"int16", "MBR_TENURE_ASOF_CK_OUT_DT":"int16", "HTL_RGN_EMEAA":"int8", "HTL_RGN_GCHINA":"int8", "HTL_CHAIN_CATEGORY_CHN_CAT_1":"int8", "HTL_CHAIN_CATEGORY_CHN_CAT_2":"int8", "HTL_CHAIN_CATEGORY_CHN_CAT_3":"int8", "REWARD_NT_RN_1":"int8", "REWARD_NT_RN_2":"int8", "HTL_CHAIN_CHN_2":"int8", "HTL_CHAIN_CHN_3":"int8", "HTL_CHAIN_CHN_4":"int8", "HTL_CHAIN_CHN_5":"int8", "HTL_CHAIN_CHN_6":"int8", "HTL_CHAIN_CHN_7":"int8", "HTL_CHAIN_CHN_9":"int8", "HTL_CHAIN_CHN_10":"int8", "HTL_CHAIN_CHN_11":"int8", "HTL_CHAIN_CHN_12":"int8", "HTL_CHAIN_CHN_13":"int8", "HTL_CHAIN_CHN_14":"int8", "HTL_CHAIN_CHN_15":"int8", "HTL_CHAIN_CHN_18":"int8", "HTL_CHAIN_CHN_20":"int8", "HTL_CHAIN_CHN_22":"int8", "HTL_CHAIN_CHN_23":"int8"}

stay_type["NUM_STAYS"] = "int16"
stay_type["TOTAL_STAY_REVENUE"] = stay_type["STAY_REVENUE_USD"]
stay_type["TOTAL_ROOMS"] = stay_type["total_rooms"]
stay_type["TOTAL_GUEST_QTY"] = stay_type["total_GUEST_QTY"]
stay_type["TOTAL_TIME_BWTN_CONF_CKIN_DT"] = stay_type["TIME_BWTN_CONF_CKIN_DT"]
stay_type["TOTAL_TIME_BWTN_CKIN_CKOUT_DT"] = stay_type["TIME_BWTN_CKIN_CKOUT_DT"]
stay_type["TOTAL_CONF_DT_daysto_holiday"] = stay_type["CONF_DT_daysto_holiday"]
stay_type["TOTAL_CK_IN_DT_daysto_holiday"] = stay_type["CK_IN_DT_daysto_holiday"]
stay_type["TOTAL_CK_OUT_DT_daysto_holiday"] = stay_type["CK_OUT_DT_daysto_holiday"]
stay_type["TOTAL_CONF_DT_nearest_holiday_dayofyear"] = stay_type["CONF_DT_nearest_holiday_dayofyear"]
stay_type["TOTAL_CK_IN_DT_nearest_holiday_dayofyear"] = stay_type["CK_IN_DT_nearest_holiday_dayofyear"]
stay_type["TOTAL_CK_OUT_DT_nearest_holiday_dayofyear"] = stay_type["CK_OUT_DT_nearest_holiday_dayofyear"]

for column_name in ['BUS_LEIS_IND', 'DIFFERENT_CONTINENT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']:
    stay_type[f"{column_name}_REVENUE"] = stay_type["STAY_REVENUE_USD"]
    stay_type[f"{column_name}_ROOMS"] = stay_type["total_rooms"]
    stay_type[f"{column_name}_GUEST_QTY"] = stay_type["total_GUEST_QTY"]
    
for column_name in ['BUS_LEIS_IND', 'DIFFERENT_CONTINENT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']:
    stay_type[f"NUM_{column_name}"] = stay_type["total_rooms"]
    stay_type[f"TOTAL_{column_name}_REVENUE"] = stay_type["STAY_REVENUE_USD"]
    stay_type[f"TOTAL_{column_name}_ROOMS"] = stay_type["total_rooms"]
    stay_type[f"TOTAL_{column_name}_GUEST_QTY"] = stay_type["total_GUEST_QTY"]
    
print(f"{datetime.datetime.now()} -- Process Started")
stay_hist_df = pd.read_csv(stay_hist_pth_process, dtype = stay_type, parse_dates = ["CONF_DT", "CK_IN_DT", "CK_OUT_DT"])

stay_hist_df.sort_values(["HASH_NBR", "CK_OUT_DT"], inplace = True)


print(f"{datetime.datetime.now()} -- Group by Member ID and Engineer Features")
stay_hist_gr = stay_hist_df.groupby("HASH_NBR")

stay_hist_df["NUM_STAYS"] = stay_hist_gr.cumcount().astype('int32')
stay_hist_df["TOTAL_STAY_REVENUE"] = stay_hist_gr["STAY_REVENUE_USD"].cumsum()
stay_hist_df["TOTAL_ROOMS"] = stay_hist_gr["total_rooms"].cumsum().astype('int32')
stay_hist_df["TOTAL_GUEST_QTY"] = stay_hist_gr["total_GUEST_QTY"].cumsum().astype('int32')

stay_hist_df["TOTAL_TIME_BWTN_CONF_CKIN_DT"] = stay_hist_gr["TIME_BWTN_CONF_CKIN_DT"].cumsum().astype('int32')
stay_hist_df["TOTAL_TIME_BWTN_CKIN_CKOUT_DT"] = stay_hist_gr["TIME_BWTN_CKIN_CKOUT_DT"].cumsum().astype('int32')

stay_hist_df["TOTAL_CONF_DT_daysto_holiday"] = stay_hist_gr["CONF_DT_daysto_holiday"].cumsum().astype('int32')
stay_hist_df["TOTAL_CK_IN_DT_daysto_holiday"] = stay_hist_gr["CK_IN_DT_daysto_holiday"].cumsum().astype('int32')
stay_hist_df["TOTAL_CK_OUT_DT_daysto_holiday"] = stay_hist_gr["CK_OUT_DT_daysto_holiday"].cumsum().astype('int32')

stay_hist_df["TOTAL_CONF_DT_nearest_holiday_dayofyear"] = stay_hist_gr["CONF_DT_nearest_holiday_dayofyear"].cumsum().astype('int32')
stay_hist_df["TOTAL_CK_IN_DT_nearest_holiday_dayofyear"] = stay_hist_gr["CK_IN_DT_nearest_holiday_dayofyear"].cumsum().astype('int32')
stay_hist_df["TOTAL_CK_OUT_DT_nearest_holiday_dayofyear"] = stay_hist_gr["CK_OUT_DT_nearest_holiday_dayofyear"].cumsum().astype('int32')

print(f"{datetime.datetime.now()} -- Further Feature Engineering")
for column_name in ['BUS_LEIS_IND', 'DIFFERENT_CONTINENT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']:
    stay_hist_df[f"{column_name}_REVENUE"] = stay_hist_df["STAY_REVENUE_USD"] * stay_hist_df[column_name]
    stay_hist_df[f"{column_name}_ROOMS"] = stay_hist_df["total_rooms"] * stay_hist_df[column_name]
    stay_hist_df[f"{column_name}_GUEST_QTY"] = stay_hist_df["total_GUEST_QTY"] * stay_hist_df[column_name]
    
print(f"{datetime.datetime.now()} -- Regrouping Data")
stay_hist_gr = stay_hist_df.groupby("HASH_NBR")

for column_name in ['BUS_LEIS_IND', 'DIFFERENT_CONTINENT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23']:
    stay_hist_df[f"NUM_{column_name}"] = (stay_hist_gr[column_name]).cumsum().astype('int32')
    stay_hist_df[f"TOTAL_{column_name}_REVENUE"] = (stay_hist_gr[f"{column_name}_REVENUE"]).cumsum()
    stay_hist_df[f"TOTAL_{column_name}_ROOMS"] = (stay_hist_gr[f"{column_name}_ROOMS"]).cumsum().astype('int32')
    stay_hist_df[f"TOTAL_{column_name}_GUEST_QTY"] = (stay_hist_gr[f"{column_name}_GUEST_QTY"]).cumsum().astype('int32')
    stay_hist_df.drop([f"{column_name}_REVENUE", f"{column_name}_ROOMS", f"{column_name}_GUEST_QTY"], axis = 1, inplace = True)
        
        
        
stay_hist_df.to_csv(stay_hist_pth_process2, index = False)

In [15]:
%%time

first_chunk = True
i = 0
em_proc_dict = {'HASH_NBR': "object", 'CAMPAIGN_NBR': "object", 'UNSUB_IND': "int8", 'SEND_DT': "object", 'CLICK': "int8", 'MBR_PRGM_ACTV': "int8", 'week_of_year': "int8", 'day_of_month': "int8", 'day_of_year': "int8", 'is_end_month': "int8", 'is_start_month': "int8", 'is_end_quarter': "int8", 'is_start_quarter': "int8", 'ENROLL_DT': "object", 'CC_2': "int8", 'CC_3': "int8", 'CC_4': "int8", 'TIER_2': "int8", 'TIER_3': "int8", 'TIER_4': "int8", 'quarter_1': "int8", 'quarter_3': "int8", 'quarter_4': "int8", 'GNDR_1': "int8", 'GNDR_2': "int8", 'EN_1': "int8", 'EN_2': "int8", 'EN_3': "int8", 'EN_4': "int8", 'EN_6': "int8", 'AGE_1': "int8", 'AGE_2': "int8", 'AGE_3': "int8", 'AGE_4': "int8", 'AGE_5': "int8", 'INCOME_1': "int8", 'INCOME_2': "int8", 'INCOME_3': "int8", 'INCOME_4': "int8", 'INCOME_5': "int8", 'January': "int8", 'February': "int8", 'April': "int8", 'May': "int8", 'June': "int8", 'July': "int8", 'August': "int8", 'September': "int8", 'October': "int8", 'November': "int8", 'December': "int8", 'Monday': "int8", 'Tuesday': "int8", 'Wednesday': "int8", 'Thursday': "int8", 'Friday': "int8", 'Saturday': "int8", 'Fall': "int8", 'Summer': "int8", 'Winter': "int8", 'Oceania': "int8", 'Asia_Africa': "int8", 'Europe': "int8", 'AUSTRALASIA_JAPAN': "int8", 'CANADA': "int8", 'CARIBBEAN': "int8", 'EUROPE': "int8", 'HONG_KONG': "int8", 'INDIA_MIDDLE_EAST_AFRICA': "int8", 'LATIN_AMERICA': "int8", 'MAINLAND_CHINA': "int8", 'MEXICO': "int8", 'SOUTHEAST_ASIA_KOREA': "int8", 'TAIWAN': "int8", 'MACAU': "int8"}
for unique_member_date_df in pd.read_csv(email_hist_pth_process, chunksize = 1000000, dtype = em_proc_dict, usecols = ["HASH_NBR", "SEND_DT"], parse_dates = ["SEND_DT"]):

    print(f"Merge:         {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    unique_member_date_df = pd.merge(left = unique_member_date_df,
                                     right = stay_hist_df,
                                     left_on = "HASH_NBR",
                                     right_on = "HASH_NBR",
                                     how = "left")
    print("#####################################################")
    print(f"Pull:          {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")
    unique_member_date_df = unique_member_date_df.apply(pull_data, axis = 1)

    print("#####################################################")
    print(f"Saving:          {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")
    if first_chunk == True:
        unique_member_date_df.to_csv(email_hist_pth_process2, index = False, mode = 'w')
        first_chunk = False
        i = i + 1
    else:
        unique_member_date_df.to_csv(email_hist_pth_process2, index = False, mode = 'a', header = False)
        i = i + 1

    print("#####################################################")
    print(f"Chunk {i} Completed: {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")

    gc.collect()
    


Merge:         2023-04-05 02:53:58.729933
% RAM memory used: 69.5 %
RAM Used (GB): 185.494298624
#####################################################
Pull:          2023-04-05 02:54:00.387055
% RAM memory used: 69.5 %
RAM Used (GB): 185.493663744
#####################################################
#####################################################
Saving:          2023-04-05 03:12:53.016919
% RAM memory used: 69.5 %
RAM Used (GB): 185.503449088
#####################################################
#####################################################
Chunk 1 Completed: 2023-04-05 03:13:12.399851
% RAM memory used: 69.5 %
RAM Used (GB): 185.504468992
#####################################################
Merge:         2023-04-05 03:13:16.654366
% RAM memory used: 69.5 %
RAM Used (GB): 185.551007744
#####################################################
Pull:          2023-04-05 03:13:18.431704
% RAM memory used: 69.5 %
RAM Used (GB): 185.550585856
##################################

########################################################################################################################################################################
########################################################################################################################################################################

# 

In [26]:
%%time
import psutil
import sqlite3
import pandas as pd

first_chunk = True
i = 0
em_proc_dict = {'HASH_NBR': "object", 'CAMPAIGN_NBR': "object", 'UNSUB_IND': "int8", 'SEND_DT': "object", 'CLICK': "int8", 'MBR_PRGM_ACTV': "int8", 'week_of_year': "int8", 'day_of_month': "int8", 'day_of_year': "int8", 'is_end_month': "int8", 'is_start_month': "int8", 'is_end_quarter': "int8", 'is_start_quarter': "int8", 'ENROLL_DT': "object", 'CC_2': "int8", 'CC_3': "int8", 'CC_4': "int8", 'TIER_2': "int8", 'TIER_3': "int8", 'TIER_4': "int8", 'quarter_1': "int8", 'quarter_3': "int8", 'quarter_4': "int8", 'GNDR_1': "int8", 'GNDR_2': "int8", 'EN_1': "int8", 'EN_2': "int8", 'EN_3': "int8", 'EN_4': "int8", 'EN_6': "int8", 'AGE_1': "int8", 'AGE_2': "int8", 'AGE_3': "int8", 'AGE_4': "int8", 'AGE_5': "int8", 'INCOME_1': "int8", 'INCOME_2': "int8", 'INCOME_3': "int8", 'INCOME_4': "int8", 'INCOME_5': "int8", 'January': "int8", 'February': "int8", 'April': "int8", 'May': "int8", 'June': "int8", 'July': "int8", 'August': "int8", 'September': "int8", 'October': "int8", 'November': "int8", 'December': "int8", 'Monday': "int8", 'Tuesday': "int8", 'Wednesday': "int8", 'Thursday': "int8", 'Friday': "int8", 'Saturday': "int8", 'Fall': "int8", 'Summer': "int8", 'Winter': "int8", 'Oceania': "int8", 'Asia_Africa': "int8", 'Europe': "int8", 'AUSTRALASIA_JAPAN': "int8", 'CANADA': "int8", 'CARIBBEAN': "int8", 'EUROPE': "int8", 'HONG_KONG': "int8", 'INDIA_MIDDLE_EAST_AFRICA': "int8", 'LATIN_AMERICA': "int8", 'MAINLAND_CHINA': "int8", 'MEXICO': "int8", 'SOUTHEAST_ASIA_KOREA': "int8", 'TAIWAN': "int8", 'MACAU': "int8"}
for partial_df in pd.read_csv(email_hist_pth_process2, chunksize = 10000000, dtype = partial_dict, parse_dates = ["SEND_DT", "CONF_DT", "CK_IN_DT", "CK_OUT_DT"]):

    print(f"Create SQLite Database:         {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    partial_df.to_sql("partial_emailstay", conn, if_exists="replace", index=False)
    
    print("#####################################################")
    print(f"Query:          {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")
    join_query = '''
                 SELECT *
                 FROM partial_emailstay
                 JOIN full_stay ON partial_emailstay.HASH_NBR = full_stay.HASH_NBR
                               AND partial_emailstay.CONF_HASH_NBR = full_stay.CONF_HASH_NBR
                               AND partial_emailstay.CONF_DT = full_stay.CONF_DT
                               AND partial_emailstay.CK_IN_DT = full_stay.CK_IN_DT
                               AND partial_emailstay.CK_OUT_DT = full_stay.CK_OUT_DT
                               AND partial_emailstay.HTL_HASH_NBR = full_stay.HTL_HASH_NBR;
                 '''

    partial_df = pd.read_sql_query(join_query, conn)
    partial_df.columns = ['HASH_NBR', 'SEND_DT', 'CONF_HASH_NBR', 'CONF_DT', 'CK_IN_DT', 'CK_OUT_DT', 'HTL_HASH_NBR', 'NUM_UNIQUE_CONF_HASH_NBR', 'NUM_UNIQUE_HTL_HASH_NBR', 'CONF_HASH_NBR_drop', 'HASH_NBR_drop', 'BUS_LEIS_IND', 'CONF_DT_drop', 'CK_IN_DT_drop', 'CK_OUT_DT_drop', 'HTL_HASH_NBR_drop', 'STAY_REVENUE_USD', 'TIME_BWTN_CONF_CKIN_DT', 'TIME_BWTN_CKIN_CKOUT_DT', 'CONF_DT_daysto_holiday', 'CK_IN_DT_daysto_holiday', 'CK_OUT_DT_daysto_holiday', 'CONF_DT_nearest_holiday_dayofyear', 'CK_IN_DT_nearest_holiday_dayofyear', 'CK_OUT_DT_nearest_holiday_dayofyear', 'DIFFERENT_CONTINENT', 'MBR_TENURE_ASOF_CONF_DT', 'MBR_TENURE_ASOF_CK_IN_DT', 'MBR_TENURE_ASOF_CK_OUT_DT', 'HTL_RGN_EMEAA', 'HTL_RGN_GCHINA', 'HTL_CHAIN_CATEGORY_CHN_CAT_1', 'HTL_CHAIN_CATEGORY_CHN_CAT_2', 'HTL_CHAIN_CATEGORY_CHN_CAT_3', 'REWARD_NT_RN_1', 'REWARD_NT_RN_2', 'HTL_CHAIN_CHN_2', 'HTL_CHAIN_CHN_3', 'HTL_CHAIN_CHN_4', 'HTL_CHAIN_CHN_5', 'HTL_CHAIN_CHN_6', 'HTL_CHAIN_CHN_7', 'HTL_CHAIN_CHN_9', 'HTL_CHAIN_CHN_10', 'HTL_CHAIN_CHN_11', 'HTL_CHAIN_CHN_12', 'HTL_CHAIN_CHN_13', 'HTL_CHAIN_CHN_14', 'HTL_CHAIN_CHN_15', 'HTL_CHAIN_CHN_18', 'HTL_CHAIN_CHN_20', 'HTL_CHAIN_CHN_22', 'HTL_CHAIN_CHN_23', 'NUM_STAYS', 'TOTAL_STAY_REVENUE', 'TOTAL_ROOMS', 'TOTAL_GUEST_QTY', 'TOTAL_TIME_BWTN_CONF_CKIN_DT', 'TOTAL_TIME_BWTN_CKIN_CKOUT_DT', 'TOTAL_CONF_DT_daysto_holiday', 'TOTAL_CK_IN_DT_daysto_holiday', 'TOTAL_CK_OUT_DT_daysto_holiday', 'TOTAL_CONF_DT_nearest_holiday_dayofyear', 'TOTAL_CK_IN_DT_nearest_holiday_dayofyear', 'TOTAL_CK_OUT_DT_nearest_holiday_dayofyear', 'NUM_BUS_LEIS_IND', 'TOTAL_BUS_LEIS_IND_REVENUE', 'TOTAL_BUS_LEIS_IND_ROOMS', 'TOTAL_BUS_LEIS_IND_GUEST_QTY', 'NUM_DIFFERENT_CONTINENT', 'TOTAL_DIFFERENT_CONTINENT_REVENUE', 'TOTAL_DIFFERENT_CONTINENT_ROOMS', 'TOTAL_DIFFERENT_CONTINENT_GUEST_QTY', 'NUM_HTL_RGN_EMEAA', 'TOTAL_HTL_RGN_EMEAA_REVENUE', 'TOTAL_HTL_RGN_EMEAA_ROOMS', 'TOTAL_HTL_RGN_EMEAA_GUEST_QTY', 'NUM_HTL_RGN_GCHINA', 'TOTAL_HTL_RGN_GCHINA_REVENUE', 'TOTAL_HTL_RGN_GCHINA_ROOMS', 'TOTAL_HTL_RGN_GCHINA_GUEST_QTY', 'NUM_HTL_CHAIN_CATEGORY_CHN_CAT_1', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_1_REVENUE', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_1_ROOMS', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_1_GUEST_QTY', 'NUM_HTL_CHAIN_CATEGORY_CHN_CAT_2', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_2_REVENUE', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_2_ROOMS', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_2_GUEST_QTY', 'NUM_HTL_CHAIN_CATEGORY_CHN_CAT_3', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_3_REVENUE', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_3_ROOMS', 'TOTAL_HTL_CHAIN_CATEGORY_CHN_CAT_3_GUEST_QTY', 'NUM_REWARD_NT_RN_1', 'TOTAL_REWARD_NT_RN_1_REVENUE', 'TOTAL_REWARD_NT_RN_1_ROOMS', 'TOTAL_REWARD_NT_RN_1_GUEST_QTY', 'NUM_REWARD_NT_RN_2', 'TOTAL_REWARD_NT_RN_2_REVENUE', 'TOTAL_REWARD_NT_RN_2_ROOMS', 'TOTAL_REWARD_NT_RN_2_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_2', 'TOTAL_HTL_CHAIN_CHN_2_REVENUE', 'TOTAL_HTL_CHAIN_CHN_2_ROOMS', 'TOTAL_HTL_CHAIN_CHN_2_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_3', 'TOTAL_HTL_CHAIN_CHN_3_REVENUE', 'TOTAL_HTL_CHAIN_CHN_3_ROOMS', 'TOTAL_HTL_CHAIN_CHN_3_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_4', 'TOTAL_HTL_CHAIN_CHN_4_REVENUE', 'TOTAL_HTL_CHAIN_CHN_4_ROOMS', 'TOTAL_HTL_CHAIN_CHN_4_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_5', 'TOTAL_HTL_CHAIN_CHN_5_REVENUE', 'TOTAL_HTL_CHAIN_CHN_5_ROOMS', 'TOTAL_HTL_CHAIN_CHN_5_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_6', 'TOTAL_HTL_CHAIN_CHN_6_REVENUE', 'TOTAL_HTL_CHAIN_CHN_6_ROOMS', 'TOTAL_HTL_CHAIN_CHN_6_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_7', 'TOTAL_HTL_CHAIN_CHN_7_REVENUE', 'TOTAL_HTL_CHAIN_CHN_7_ROOMS', 'TOTAL_HTL_CHAIN_CHN_7_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_9', 'TOTAL_HTL_CHAIN_CHN_9_REVENUE', 'TOTAL_HTL_CHAIN_CHN_9_ROOMS', 'TOTAL_HTL_CHAIN_CHN_9_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_10', 'TOTAL_HTL_CHAIN_CHN_10_REVENUE', 'TOTAL_HTL_CHAIN_CHN_10_ROOMS', 'TOTAL_HTL_CHAIN_CHN_10_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_11', 'TOTAL_HTL_CHAIN_CHN_11_REVENUE', 'TOTAL_HTL_CHAIN_CHN_11_ROOMS', 'TOTAL_HTL_CHAIN_CHN_11_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_12', 'TOTAL_HTL_CHAIN_CHN_12_REVENUE', 'TOTAL_HTL_CHAIN_CHN_12_ROOMS', 'TOTAL_HTL_CHAIN_CHN_12_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_13', 'TOTAL_HTL_CHAIN_CHN_13_REVENUE', 'TOTAL_HTL_CHAIN_CHN_13_ROOMS', 'TOTAL_HTL_CHAIN_CHN_13_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_14', 'TOTAL_HTL_CHAIN_CHN_14_REVENUE', 'TOTAL_HTL_CHAIN_CHN_14_ROOMS', 'TOTAL_HTL_CHAIN_CHN_14_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_15', 'TOTAL_HTL_CHAIN_CHN_15_REVENUE', 'TOTAL_HTL_CHAIN_CHN_15_ROOMS', 'TOTAL_HTL_CHAIN_CHN_15_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_18', 'TOTAL_HTL_CHAIN_CHN_18_REVENUE', 'TOTAL_HTL_CHAIN_CHN_18_ROOMS', 'TOTAL_HTL_CHAIN_CHN_18_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_20', 'TOTAL_HTL_CHAIN_CHN_20_REVENUE', 'TOTAL_HTL_CHAIN_CHN_20_ROOMS', 'TOTAL_HTL_CHAIN_CHN_20_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_22', 'TOTAL_HTL_CHAIN_CHN_22_REVENUE', 'TOTAL_HTL_CHAIN_CHN_22_ROOMS', 'TOTAL_HTL_CHAIN_CHN_22_GUEST_QTY', 'NUM_HTL_CHAIN_CHN_23', 'TOTAL_HTL_CHAIN_CHN_23_REVENUE', 'TOTAL_HTL_CHAIN_CHN_23_ROOMS', 'TOTAL_HTL_CHAIN_CHN_23_GUEST_QTY']
    partial_df.drop([x for x in partial_df.columns.to_list() if "_drop" in x], axis = 1, inplace = True)

    print("#####################################################")
    print(f"Saving:          {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")
    if first_chunk == True:
        partial_df.to_csv(email_hist_pth_process3, index = False, mode = 'w')
        first_chunk = False
        i = i + 1
    else:
        partial_df.to_csv(email_hist_pth_process3, index = False, mode = 'a', header = False)
        i = i + 1

    print("#####################################################")
    print(f"Chunk {i} Completed: {datetime.datetime.now()}")
    print(f'% RAM memory used: {psutil.virtual_memory()[2]} %')
    print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    print("#####################################################")

    gc.collect()

Create SQLite Database:         2023-04-08 01:00:13.936137
% RAM memory used: 79.8 %
RAM Used (GB): 213.167562752
#####################################################
Query:          2023-04-08 01:04:33.869092
% RAM memory used: 79.8 %
RAM Used (GB): 213.25815808
#####################################################
#####################################################
Saving:          2023-04-08 01:17:35.608934
% RAM memory used: 83.2 %
RAM Used (GB): 222.323089408
#####################################################
#####################################################
Chunk 1 Completed: 2023-04-08 01:20:51.374432
% RAM memory used: 83.2 %
RAM Used (GB): 222.320898048
#####################################################
Create SQLite Database:         2023-04-08 01:21:21.665644
% RAM memory used: 81.2 %
RAM Used (GB): 217.090269184
#####################################################
Query:          2023-04-08 01:25:35.422477
% RAM memory used: 81.2 %
RAM Used (GB): 217.093455872

############################################################################################################################################################################################################################

############################################################################################################################################################################################################################